In [1]:
devtools::install_github("clairbarnes/rwwa")
library(rwwa)
library(ellipse)

In [ ]:
gmst <- read.fwf("gmst.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2021]

df <- merge(gmst, read.fwf("maxtmax_ghcn-portland.dat", comment.char = "#", widths = c(5,15), col.names = c("year", "tmax")))
mdl <- fit_ns("gev", "shift", df, "tmax", "gmst", lower = F, ev = df$tmax[df$year == 2021])

cov <- 0
cov_cf <- -1.2
ev <- df$tmax[df$year == 2021]

pars_2021 <- ns_pars(mdl, fixed_cov = cov)


In [114]:
# get confidence interval for mu'
set.seed(42)
mu_ci <- apply(sapply(1:50, function(i) {
    boot_df <- df[sample(1:nrow(df), nrow(df), replace = T),]
    tryCatch({
        boot_mdl <- fit_ns(mdl$dist, mdl$type, boot_df, varnm = mdl$varnm, covnm_1 = mdl$covnm_1, lower = mdl$lower, mintemps = mdl$mintemps, ev = ev)
        c("mu_ev" = ns_pars(boot_mdl, fixed_cov = cov)$loc,
          "mu_cf" = ns_pars(boot_mdl, fixed_cov = cov_cf)$loc)
    }, error = function(cond) {return(rep(NA, 2))})
}), 1, quantile, c(0.025, 0.975), na.rm = T)

In [387]:
prep_window(c(1,3))

png("fig_wcrp-statistics.png", height = 480, width = 480 * 3); par(mfrow = c(1,3), cex = 1.4, mar = c(4,4,2,0)); {
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # GMST trend

    plot(mdl$cov1, mdl$x, pch = 20, main = "", xlab = "", ylab = "", ylim = c(30,50), xlim = range(c(mdl$cov1, cov, cov_cf)))
    mtext(expression("GMST anomaly wrt 2021 ("*degree*"C)"), side = 1, line = 2.5, cex = par("cex"))
    mtext(expression("Maximum temperature ("*degree*"C)"), side = 2, line = 2.5, cex = par("cex"))
    mtext("(a)", side = 3, line = 0.5, cex = par("cex"), adj = 0, font = 2)

    points(cov, ev, col = "magenta", lwd = 2, pch = 0)

    # trend lines
    lines(mdl$cov1, ns_pars(mdl)$loc, lwd = 3, col = "black", lty = 1)
    lines(mdl$cov1, eff_return_level(6, mdl), col = "blue", lwd = 2, lty = 1)
    lines(mdl$cov1, eff_return_level(40, mdl), col = "blue", lwd = 1, lty = 1)

    # confidence interval & markers for mu' at factual & counterfactual covariates
    lines(rep(cov, 3), c(ns_pars(mdl, fixed_cov = cov)$loc, mu_ci[,"mu_ev"]), col = "black", lwd = 3, type = "o", pch = "_", cex = 1.3)
    lines(rep(cov_cf, 3), c(ns_pars(mdl, fixed_cov = cov_cf)$loc, mu_ci[,"mu_cf"]), col = "black", lwd = 3, type = "o", pch = "_", cex = 1.3)

    # add legend
    legend("topleft", legend = c("location", "1-in-6-year event", "1-in-40-year event"), lty = 1, col = c("black", "blue", "blue"), lwd = c(2,2,1), bty = "n")

    # lines(df$gmst[order(df$gmst)], filter(df$tmax[order(df$gmst)], rep(1/15, 15), sides = 1), col = "forestGreen", lwd = 3)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Schematic of pdfs
    
    obs <- 43
    x <- seq(28,49,0.1)
    xx <- seq(obs,49,0.01)
    
    dns_2021 <- function(x) devd(x, loc = pars_2021$loc, shape = pars_2021$shape, scale = pars_2021$scale)
    dns_hist <- function(x) devd(x, loc = pars_2021$loc-2, shape = pars_2021$shape, scale = pars_2021$scale)
        
    plot(0, type = "n", xaxs = "i", yaxs = "i", xaxt = "n", yaxt = "n", bty = "n", xlim = c(min(x), max(x)), ylim = c(0,0.2), xlab = "", ylab = "")
    ymax <- par("usr")[4]*0.85
    
    # axes & axis labels
    mtext(expression("Temperature"), side = 1, line = 0.5, cex = 1.5)
    mtext("Probability density", side = 2, cex = 1.5, line = 0.5)
    mtext("(b)", side = 3, line = 0.5, cex = par("cex"), adj = 0, font = 2)
    
    # add lines showing counterfactual & factual densities
    lines(x, dns_2021(x), type = "l", lwd = 3, col = "black")
    lines(x, dns_hist(x), type = "l", lwd = 3, col = "black", lty = 3)
    
    abline(h = 0, v = min(x), lwd = 3)
    # Arrows(min(x),0,min(x),ymax, lwd = 3)
    # Arrows(min(x),0,max(x)-0.2,0, lwd = 3)
    
    # line marking observed temperature
    lines(rep(obs,2), c(0,ymax), col = "dimgrey", lty = 2, lwd = 3)
    # text(obs, ymax, "Observed", col = "dimgrey", pos = 3)
    
    # add shaded polygons
    polygon(c(xx, rev(xx)), c(dns_2021(xx), rep(0,length(xx))), col = adjustcolor("red", alpha = 0.3), border = NA)
    polygon(c(xx, rev(xx)), c(rep(0,length(xx)), rev(dns_hist(xx))), col = adjustcolor("blue", alpha = 0.3), border = NA)
    
    # add labels (will need to play around with x & y coordinates to get best location)
    text(45,.055, "p1", font = 2)
    lines(c(44.75, 44),c(.05, .02), lwd = 3)
    text(46,.02, "p0", font = 2)
    lines(c(45.3, 43.5),c(.02, .0051), lwd = 3)
    
    # add label for change in intensity
    max_dns <- max(dns_2021(x))
    Arrows(x[which(dns_hist(x) == max_dns)], max_dns+.0025, x[which(dns_2021(x) == max_dns)], max_dns+.0025, arr.adj = 1, lwd = 3)
    text(39.5, max_dns+.01, expression(Delta*"I"), font = 2)
    
    legend("topleft", legend = c("Present climate", "Past climate"), lty = c(1,3), lwd = 3, bty = "n")

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Schematic of joint pdfs
    
    x_obs <- 1
    y_obs <- -2
    
    ell_cf <- ellipse(-0.7, center = c(-0.8,0), npoints = 200)
    ell_f <- ellipse(-0.7, center = c(0,-0.1), npoints = 300)
    
    xy_f <- ell_f[(ell_f[,"x"] >= x_obs) & (ell_f[,"y"] <= y_obs),]
    xy_cf <- ell_cf[(ell_cf[,"x"] >= x_obs) & (ell_cf[,"y"] <= y_obs),]

    par(mar = c(4,4,2,1))
    x <- y <- seq(-4,4,0.01)
    plot(0, type = "n", xaxs = "i", yaxs = "i", xaxt = "n", yaxt = "n", bty = "n", xlim = range(x), ylim = range(y), xlab = "", ylab = "")

    # axes & axis labels
    mtext(expression("Temperature"), side = 1, line = 0.5, cex = 1.5)
    mtext("Precipitation", side = 2, cex = 1.5, line = 0.5)
    mtext("(c)", side = 3, line = 0.5, cex = par("cex"), adj = 0, font = 2)
    abline(h = min(y), v = min(x), lwd = 3)
    
    lines(ell_f, type = "l", lwd = 3, lty = 1, col = "black")
    lines(ell_cf, type = "l", lwd = 3, lty = 2, col = "black")
    
    lines(rep(x_obs,2), c(y_obs,-5), col = "dimgrey", lty = 2, lwd = 3)
    lines(c(x_obs,5), rep(y_obs,2), col = "dimgrey", lty = 2, lwd = 3)
    
    polygon(c(xy_f[,"x"], rev(xy_f[,"x"])), c(xy_f[,"y"], rep(y_obs, nrow(xy_f))), col = adjustcolor("red", 0.3), border = NA)
    polygon(c(xy_cf[,"x"], rev(xy_cf[,"x"])), c(xy_cf[,"y"], rep(y_obs, nrow(xy_cf))), col = adjustcolor("blue", 0.3), border = NA)
    
    Arrows(-0.8,0,0,-0.1, arr.adj = 1, lwd = 3)
    text(-0.5,0.25, expression(Delta*"I"), font = 2)
    
    text(1.5,-3.5, "p0", font = 2)
    lines(c(1.3, 1.2),c(-3.25, -2.2), lwd = 3)
    text(2.5,-3, "p1", font = 2)
    lines(c(2.3, 2),c(-2.75,-2.2), lwd = 3)
    
    legend("topright", legend = c("Present climate", "Past climate"), lty = c(1,3), lwd = 3, bty = "n")

}; dev.off()

png 
  2